In [ ]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-storage.json'

In [ ]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-general')

In [ ]:
!mkdir t5-base-bahasa

In [ ]:
blob = bucket.blob('t5-base/model.ckpt-166300.data-00000-of-00002')
blob.download_to_filename('t5-base-bahasa/model.ckpt-166300.data-00000-of-00002')

In [ ]:
blob = bucket.blob('t5-base/model.ckpt-166300.data-00001-of-00002')
blob.download_to_filename('t5-base-bahasa/model.ckpt-166300.data-00001-of-00002')

In [ ]:
blob = bucket.blob('t5-base/model.ckpt-166300.index')
blob.download_to_filename('t5-base-bahasa/model.ckpt-166300.index')

In [ ]:
blob = bucket.blob('t5-base/model.ckpt-166300.meta')
blob.download_to_filename('t5-base-bahasa/model.ckpt-166300.meta')

In [ ]:
blob = bucket.blob('t5-base/checkpoint')
blob.download_to_filename('t5-base-bahasa/checkpoint')

In [ ]:
blob = bucket.blob('t5-base/operative_config.gin')
blob.download_to_filename('t5-base-bahasa/operative_config.gin')

In [1]:
# !pip3 install transformers -U
from transformers import T5Config, T5Model, load_tf_weights_in_t5

In [2]:
import os

out = 't5-base-bahasa-cased'
os.makedirs(out, exist_ok=True)

In [3]:
config = T5Config(
    vocab_size = 32128,
    n_positions=1024,
    d_ff = 3072,
    d_kv = 64,
    d_model = 768,
    dropout_rate = 0.1,
    inputs_length = 1024,
    num_heads = 12,
    num_layers = 12,
    decoder_start_token_id = 0,
    eos_token_id = 1,
    pad_token_id = 0)
print(config)
config.save_pretrained(out)

T5Config {
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "inputs_length": 1024,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 1024,
  "num_heads": 12,
  "num_layers": 12,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "vocab_size": 32128
}



In [4]:
model = T5Model(config)
load_tf_weights_in_t5(model, config, 't5-base-bahasa/model.ckpt-166300')

T5Model(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, inplac

In [5]:
from transformers import CONFIG_NAME, WEIGHTS_NAME
CONFIG_NAME, WEIGHTS_NAME

('config.json', 'pytorch_model.bin')

In [6]:
import torch

torch.save(model.state_dict(), out + '/' + WEIGHTS_NAME)

In [7]:
from transformers import T5Config, T5Model, T5Tokenizer

In [8]:
tokenizer = T5Tokenizer('sp10m.cased.t5.model')
tokenizer.save_pretrained(out)

('t5-base-bahasa-cased/spiece.model',
 't5-base-bahasa-cased/special_tokens_map.json',
 't5-base-bahasa-cased/added_tokens.json')

In [9]:
tokenizer = T5Tokenizer.from_pretrained('./t5-base-bahasa-cased', lower = False)

In [10]:
config = T5Config.from_pretrained('./t5-base-bahasa-cased')

In [11]:
model = T5Model.from_pretrained('./t5-base-bahasa-cased/pytorch_model.bin', config = config)

In [13]:
model.save_pretrained(out)

In [14]:
# !transformers-cli upload ./t5-base-bahasa-cased

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [23]:
tokenizer = T5Tokenizer.from_pretrained('huseinzol05/t5-base-bahasa-cased')
model = T5ForConditionalGeneration.from_pretrained('huseinzol05/t5-base-bahasa-cased')
input_ids = tokenizer.encode('soalan: siapakah perdana menteri malaysia?', return_tensors = 'pt')
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0])

'Mahathir Mohamad'